# Scripts

In [14]:
import requests
from bs4 import BeautifulSoup

In [20]:
def obtener_contenido_pagina(url):
    response = requests.get(url)
    return response.content

In [21]:
def analizar_contenido_html(html):
    return BeautifulSoup(html, 'html.parser')

In [24]:
data = []

def procesar_pagina(soup):

    titulos = []
    precios_originales = []
    precios_descuento = []
    precios_envio = []
    skus = []
    stocks = []

    titulo_items = soup.find_all('div', class_="emproduct_right")

    for item in titulo_items:
        titulo = item.a.text.strip()
        titulos.append(titulo)

    precio_original_items = soup.find_all("span", class_="oldPrice")

    for item in precio_original_items:
        precio_original = item.text.strip()
        precios_originales.append(precio_original)

    precio_descuento_items = soup.find_all("div", class_="emproduct_right_price_left")

    for item in precio_descuento_items:
        precio_descuento = item.label.text.strip()
        precios_descuento.append(precio_descuento)

    sku_items = soup.find_all("div", class_="emproduct_right_artnum")

    for item in sku_items:
        sku = item.text.strip()
        skus.append(sku.lstrip("SKU: "))

    precio_envio_items = soup.find_all("span", class_={"deliveryvalue", "deliverytextfree"})

    for item in precio_envio_items:
        precio_envio = item.text
        precios_envio.append(precio_envio)

    stock_items = soup.find_all("div", class_="emstock")

    for item in stock_items:
        stock = item.text.split()
        stocks.append(stock[1])

    min_length = min(len(titulos), len(skus), len(precios_originales), len(precios_descuento), len(precios_envio), len(stocks))
    
    for i in range(min_length):
        data.append({
            "Título": titulos[i] if i < len(titulos) else None,
            "SKU": skus[i]  if i < len(skus) else None,
            "Precio Original": precios_originales[i]  if i < len(precios_originales) else None,
            "Precio Descuento": precios_descuento[i] if i < len(precios_descuento) else None,
            "Precio Envío": precios_envio[i] if i < len(precios_envio) else None,
            "Stock": stocks[i] if i < len(stocks) else None
        })


In [25]:
def manejar_paginacion(url_base, num_paginas):
    for i in range(1, num_paginas + 1):
        url = url_base + str(i) + "/"
        contenido_pagina = obtener_contenido_pagina(url)
        soup = analizar_contenido_html(contenido_pagina)
        procesar_pagina(soup)



In [26]:
url_base = "https://www.cyberpuerta.mx/Promociones/"
num_paginas = 10

manejar_paginacion(url_base, num_paginas)

In [27]:
import pandas as pd
df = pd.DataFrame(data)
df

,Título,SKU,Precio Original,Precio Descuento,Precio Envío,Stock
0,"SSD Externo Adata SE880, 1TB, USB-C 3.2, Gris",AELI-SE880-1TCGY,"$1,197.00","$1,079.00",$99.00,38
1,Router ASUS Gigabit Ethernet de Banda Dual RT-...,90IG0300-BA1000,"$1,859.00","$1,009.00",Envío gratis,201
2,"Cartucho Brother LC107BK Negro, 1200 Páginas",LC107BK,$562.00,$168.00,$99.00,¡1
3,"Nexxt Solutions Foco LED Inteligente NHB-W310,...",NHB-W310 3PK,"$1,019.00",$381.00,$99.00,20
4,"X-Case Caja ACCREDKE10, 1x RJ-45, 10 Piezas",ACCREDKE10,$365.00,$51.00,$99.00,5
...,...,...,...,...,...,...
155,"Multifuncional Xerox B235, Blanco y Negro, Lás...",B235/DNI,"$6,179.00","$4,669.00",$164.00,88
156,"SSD Western Digital WD Blue SN570 NVMe, 250GB,...",WDS250G3B0C,$546.00,$420.00,$99.00,7
157,Router Tenda Gigabit Ethernet de Banda Dual Fi...,AC8,$639.00,$440.00,$99.00,5
158,"SSD Kingston FURY Renegade NVMe, 2TB, PCI Expr...",FYRD/2000G,"$2,715.00","$2,319.00",$99.00,24


In [8]:
import datetime
fecha_actual = datetime.datetime.now().strftime("%d-%m-%Y")

df.to_csv(f"./dataset/Cyberpuerta-WebScrap-{fecha_actual}.csv",index=False)